In [1]:
from pathlib import Path
import json

In [3]:
file = Path("../arc-prize-2025/arc-agi_training_challenges.json")

data = json.load(file.open())

In [ ]:
import numpy as np

for k, v in data.items():
    print("challenge", k)
    # pretty pre
    for example in v["train"]:
        input = example["input"]
        output = example["output"]
        print(np.array(input))
        print(np.array(output))
        break
    break


challenge 00576224
[[7 9]
 [4 3]]
[[7 9 7 9 7 9]
 [4 3 4 3 4 3]
 [9 7 9 7 9 7]
 [3 4 3 4 3 4]
 [7 9 7 9 7 9]
 [4 3 4 3 4 3]]


In [ ]:
from arc_competition.task_viewer import view_training_challenge

# Quick view of a training challenge
view_training_challenge("00576224")

# # Or with custom file path
# from pathlib import Path

# html_view = task_viewer("arc-prize-2025/arc-agi_training_challenges.json", "00576224")
# display(html_view)

In [ ]:
from arc_competition.task_viewer import ARCChallenge, ARCDataLoader, ROOT_DIR

file_path = ROOT_DIR / "arc-prize-2025" / "arc-agi_training_challenges.json"

challenge = ARCDataLoader.load_challenge(file_path, "00576224")

In [2]:
from arc_competition.agents import create_grid_analysis_agent
from arc_competition.schemas import GridAnalysisInput

agent = create_grid_analysis_agent()

user_input = GridAnalysisInput(
    input_grid=challenge.train[0].input,
    output_grid=challenge.train[0].output,
)
output = agent.run(user_input)


In [ ]:
user_input

ARCChallenge(train=[GridExample(input=[[7, 9], [4, 3]], output=[[7, 9, 7, 9, 7, 9], [4, 3, 4, 3, 4, 3], [9, 7, 9, 7, 9, 7], [3, 4, 3, 4, 3, 4], [7, 9, 7, 9, 7, 9], [4, 3, 4, 3, 4, 3]]), GridExample(input=[[8, 6], [6, 4]], output=[[8, 6, 8, 6, 8, 6], [6, 4, 6, 4, 6, 4], [6, 8, 6, 8, 6, 8], [4, 6, 4, 6, 4, 6], [8, 6, 8, 6, 8, 6], [6, 4, 6, 4, 6, 4]])], test=[GridExample(input=[[3, 2], [7, 8]], output=[])])

In [26]:
from atomic_agents import AgentConfig, AtomicAgent
from atomic_agents.context import SystemPromptGenerator
from arc_competition.config import config
from pydantic import Field
from atomic_agents.base.base_io_schema import BaseIOSchema
from pydantic import BaseModel
from arc_competition.task_viewer import GridExample
from arc_competition.task_viewer import Grid
from arc_competition.utils import calculate_score, calculate_difference_grid


class GridExampleModifiedInput(GridExample):
    """Extends schema for GridExample."""

    current: Grid = Field(description="Current grid as 2D list of integers")
    difference: Grid = Field(description="Difference grid as 2D list of 1s and 0s")
    score: float = Field(description="Score of the current grid")


# Grid Analysis Schemas
class GridAnalysisInput(BaseIOSchema):
    """Input schema for ARC grid analysis."""

    example_grids: list[GridExampleModifiedInput] = Field(
        description="Example grids containing the input and output grids"
    )


class GridAnalysisOutput(BaseIOSchema):
    """Output schema for ARC grid analysis."""

    # Overall summary
    analysis_summary: str = Field(description="Overall summary of the analysis")

    function_to_execute: str = Field(description="python function to execute")


def create_generator_agent() -> AtomicAgent[GridAnalysisInput, GridAnalysisOutput]:
    """Create a grid pattern analysis agent specialized for ARC challenges."""

    client = config.client
    # Create optimized system prompt generator for grid analysis
    system_prompt_generator = SystemPromptGenerator(
        background=[
            "You are an expert in pattern recognition and visual reasoning, specialized in analyzing grid-based puzzles.",
            "You have deep expertise in ARC (Abstraction and Reasoning Corpus) challenges and can identify subtle patterns.",
            "You understand both low-level pixel patterns and high-level conceptual transformations.",
            "Given an input grid, current grid and an output grid, you can identify the next best function to execute to transform the current grid into the output grid.",
            "Only use small python functions, preferably less than 10 lines of code but no more than 50 lines of code.",
            "You will also be given a score for the current grid, you can use this to identify issues with the functions or the order of execution.",
            "The score is a float between 0 and 100, where 0 is the worst and 100 is the best.",
            "50 points are deducted for mismatch output sizes between the current grid and the output grid.",
            "1 point is deducted for each element in the current grid that is unmatched in the output grid.",
            "all comparisons will be done with the top left element of the current grid as the reference point.",
            "a reference difference grid will be provided to you, you can use this to identify the differences between the current grid and the output grid.",
            "The reference difference grid will be a 2D grid of 0s and 1s, where 0s are the elements that are matched and 1s are the elements that are unmatched.",
        ]
    )

    # Create the grid analysis agent
    grid_agent = AtomicAgent[GridAnalysisInput, GridAnalysisOutput](
        config=AgentConfig(
            client=client,
            model=config.model.name,
            system_prompt_generator=system_prompt_generator,
            model_api_parameters=config.model.get_api_parameters(),
        )
    )

    return grid_agent


def update_to_modified_input(input: Grid, output: Grid) -> GridExampleModifiedInput:
    return GridExampleModifiedInput(
        input=input,
        output=output,
        current=input,
        score=calculate_score(input, output),
        difference=calculate_difference_grid(input, output),
    )


examples = [
    update_to_modified_input(example.input, example.output)
    for example in challenge.train
]

user_input = GridAnalysisInput(
    example_grids=examples,
)

grid_agent = create_generator_agent()
output = grid_agent.run(user_input)

In [29]:
print(output.function_to_execute)

def expand_and_repeat_grid(grid):
    # Repeat horizontally
    expanded_row1 = grid[0] * 3
    expanded_row2 = grid[1] * 3
    
    # Create swapped rows
    swapped_row1 = grid[1] + grid[0] + grid[1]
    swapped_row2 = grid[0] + grid[1] + grid[0]
    
    # Combine all rows
    return [
        expanded_row1,
        expanded_row2,
        swapped_row1,
        swapped_row2,
        expanded_row1,
        expanded_row2
    ]


In [ ]:
def expand_and_repeat_grid(grid):
    # Repeat horizontally
    expanded_row1 = grid[0] * 3
    expanded_row2 = grid[1] * 3

    # Create swapped rows
    swapped_row1 = grid[1] + grid[0] + grid[1]
    swapped_row2 = grid[0] + grid[1] + grid[0]

    # Combine all rows
    return [
        expanded_row1,
        expanded_row2,
        swapped_row1,
        swapped_row2,
        expanded_row1,
        expanded_row2,
    ]


llm_output = expand_and_repeat_grid(challenge.train[0].input)
llm_output = Grid(llm_output)
diff_grid = calculate_difference_grid(llm_output, challenge.train[0].output)
calculate_score(llm_output, challenge.train[0].output)

np.float64(88.0)

In [34]:
diff_grid

[[0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0]]

In [35]:
challenge.train[0].output

[[7, 9, 7, 9, 7, 9],
 [4, 3, 4, 3, 4, 3],
 [9, 7, 9, 7, 9, 7],
 [3, 4, 3, 4, 3, 4],
 [7, 9, 7, 9, 7, 9],
 [4, 3, 4, 3, 4, 3]]

In [36]:
llm_output

[[7, 9, 7, 9, 7, 9],
 [4, 3, 4, 3, 4, 3],
 [4, 3, 7, 9, 4, 3],
 [7, 9, 4, 3, 7, 9],
 [7, 9, 7, 9, 7, 9],
 [4, 3, 4, 3, 4, 3]]

In [12]:
print(output.function_to_execute)

def expand_and_alternate(grid):
    rows, cols = len(grid), len(grid[0])
    new_rows, new_cols = rows * 3, cols * 3
    new_grid = [[0 for _ in range(new_cols)] for _ in range(new_rows)]
    
    for i in range(new_rows):
        for j in range(new_cols):
            if i % 4 < 2:
                new_grid[i][j] = grid[i % 2][j % cols]
            else:
                new_grid[i][j] = grid[(i + 1) % 2][j % cols]
    
    return new_grid


In [6]:
print(output.model_dump_json(indent=4))

{
    "analysis_summary": "The transformation from the input grid to the output grid involves a pattern of repetition and expansion. The 2x2 input grid is used as a template to create a 6x6 output grid through horizontal and vertical repetition, with an additional alternating pattern in the odd-numbered rows.",
    "detected_patterns": [
        "Horizontal repetition of the input grid",
        "Vertical repetition of the input grid",
        "Alternating pattern in odd-numbered rows",
        "Preservation of original number positions",
        "3x expansion in both dimensions"
    ],
    "overall_solution_confidence": 0.95
}


In [2]:
print(challenge.train[0].prompt_format())

Input:
7 9
4 3

Output:
7 9 7 9 7 9
4 3 4 3 4 3
9 7 9 7 9 7
3 4 3 4 3 4
7 9 7 9 7 9
4 3 4 3 4 3
